<a href="https://colab.research.google.com/github/AI4Finance-LLC/FinRL-Library/blob/master/Crypto_Binance_Historical_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fetch historical data

Python script to fetch historical data from binance using ccxt

In [1]:
# Install openpyxl and CCXT
!pip install openpyxl ccxt

     |################################| 2.0 MB 3.9 MB/s eta 0:00:01
     |################################| 293 kB 17.9 MB/s eta 0:00:01
     |################################| 1.3 MB 15.2 MB/s eta 0:00:01
     |################################| 141 kB 11.2 MB/s eta 0:00:01
     |################################| 291 kB 9.0 MB/s eta 0:00:01
     |################################| 243 kB 19.5 MB/s eta 0:00:01
     |################################| 78 kB 2.8 MB/s  eta 0:00:01
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-py3-none-any.whl size=3161 sha256=60db227d2e3ff4e52393ec528a08fa6b7812bd859d678cfa2c7fac015ba89154
  Stored in directory: /root/.cache/pip/wheels/6a/f5/9c/f8331a854f7a8739cf0e74c13854e4dd7b1af11b04fe1dde13
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26308 sha256=03a802f7f03273e6d21648ef41982c65c182ee8fcd7cffb65bde16897667d5e2
  Stored in directory: /root/.cache/pip/wheels/5f/63/c2/b85489bbea28cb5d36cfe197244f898428004fa3caa7a23116
S

In [9]:
import os
from pathlib import Path

import sys
import csv
import pandas as pd

# -----------------------------------------------------------------------------

root = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(''))))
sys.path.append(root + '/python')

import ccxt


# -----------------------------------------------------------------------------

def retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    num_retries = 0
    try:
        num_retries += 1
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        # print('Fetched', len(ohlcv), symbol, 'candles from', exchange.iso8601 (ohlcv[0][0]), 'to', exchange.iso8601 (ohlcv[-1][0]))
        return ohlcv
    except Exception:
        if num_retries > max_retries:
            raise  # Exception('Failed to fetch', timeframe, symbol, 'OHLCV in', max_retries, 'attempts')


def scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    earliest_timestamp = exchange.milliseconds()
    timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe)
    timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
    timedelta = limit * timeframe_duration_in_ms
    all_ohlcv = []
    while True:
        fetch_since = earliest_timestamp - timedelta
        ohlcv = retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, fetch_since, limit)
        # if we have reached the beginning of history
        if ohlcv[0][0] >= earliest_timestamp:
            break
        earliest_timestamp = ohlcv[0][0]
        all_ohlcv = ohlcv + all_ohlcv
        print(len(all_ohlcv), symbol, 'candles in total from', exchange.iso8601(all_ohlcv[0][0]), 'to', exchange.iso8601(all_ohlcv[-1][0]))
        # if we have reached the checkpoint
        if fetch_since < since:
            break
    return all_ohlcv


def write_to_csv(filename, exchange, data):
    p = Path("./data/raw/", str(exchange))
    p.mkdir(parents=True, exist_ok=True)
    full_path = p / str(filename)
    with Path(full_path).open('w+', newline='') as output_file:
        csv_writer = csv.writer(output_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerows(data)


def scrape_candles_to_csv(filename, exchange_id, max_retries, symbol, timeframe, since, limit):
    # instantiate the exchange by id
    exchange = getattr(ccxt, exchange_id)({
        'enableRateLimit': True,  # required by the Manual
    })
    # convert since from string to milliseconds integer if needed
    if isinstance(since, str):
        since = exchange.parse8601(since)
    # preload all markets from the exchange
    exchange.load_markets()
    # fetch all candles
    ohlcv = scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit)
    # save them to csv file
    write_to_csv(filename, exchange, ohlcv)
    print('Saved', len(ohlcv), 'candles from', exchange.iso8601(ohlcv[0][0]), 'to', exchange.iso8601(ohlcv[-1][0]), 'to', filename)
    
    
def scrape_candles_to_df(filename, exchange_id, max_retries, symbol, timeframe, since, limit):
    # instantiate the exchange by id
    exchange = getattr(ccxt, exchange_id)({
        'enableRateLimit': True,  # required by the Manual
    })
    # convert since from string to milliseconds integer if needed
    if isinstance(since, str):
        since = exchange.parse8601(since)
    # preload all markets from the exchange
    exchange.load_markets()
    # fetch all candles
    ohlcv = scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit)
    # save them to df
    data_df = pd.DataFrame(data=ohlcv)
    return data_df

In [10]:
data = scrape_candles_to_df('btc_usdt_1m.csv', 'binance', 3, 'BTC/USDT', '1m', '2021-05-0100:00:00Z', 1000)
# scrape_candles_to_csv('./data/raw/binance/eth_btc_1m.csv', 'binance', 3, 'ETH/BTC', '1m', '2018-01-01T00:00:00Z', 1000)
# scrape_candles_to_csv('./data/raw/binance/ltc_btc_1m.csv', 'binance', 3, 'LTC/BTC', '1m', '2018-01-01T00:00:00Z', 1000)
# scrape_candles_to_csv('./data/raw/binance/xlm_btc_1m.csv', 'binance', 3, 'XLM/BTC', '1m', '2018-01-01T00:00:00Z', 1000)
data.head()

1000 BTC/USDT candles in total from 2021-05-30T04:31:00.000Z to 2021-05-30T21:10:00.000Z
2000 BTC/USDT candles in total from 2021-05-29T11:51:00.000Z to 2021-05-30T21:10:00.000Z
3000 BTC/USDT candles in total from 2021-05-28T19:11:00.000Z to 2021-05-30T21:10:00.000Z
4000 BTC/USDT candles in total from 2021-05-28T02:31:00.000Z to 2021-05-30T21:10:00.000Z
5000 BTC/USDT candles in total from 2021-05-27T09:51:00.000Z to 2021-05-30T21:10:00.000Z
6000 BTC/USDT candles in total from 2021-05-26T17:11:00.000Z to 2021-05-30T21:10:00.000Z
7000 BTC/USDT candles in total from 2021-05-26T00:31:00.000Z to 2021-05-30T21:10:00.000Z
8000 BTC/USDT candles in total from 2021-05-25T07:51:00.000Z to 2021-05-30T21:10:00.000Z
9000 BTC/USDT candles in total from 2021-05-24T15:11:00.000Z to 2021-05-30T21:10:00.000Z
10000 BTC/USDT candles in total from 2021-05-23T22:31:00.000Z to 2021-05-30T21:10:00.000Z
11000 BTC/USDT candles in total from 2021-05-23T05:51:00.000Z to 2021-05-30T21:10:00.000Z
12000 BTC/USDT cand

,0,1,2,3,4,5
0,1619769060000,54283.73,54286.00,54261.42,54267.52,25.456899
1,1619769120000,54267.51,54273.36,54263.74,54264.47,25.162633
2,1619769180000,54264.48,54272.67,54261.44,54268.82,19.513919
3,1619769240000,54268.82,54304.21,54264.00,54280.33,34.360138
4,1619769300000,54280.32,54304.20,54275.15,54291.79,29.218204


In [11]:
data.columns = [
                "date",
                "open",
                "high",
                "low",
                "close",
             
                "volume",
          
            ]

In [12]:
data.head()

,date,open,high,low,close,volume
0,1619769060000,54283.73,54286.00,54261.42,54267.52,25.456899
1,1619769120000,54267.51,54273.36,54263.74,54264.47,25.162633
2,1619769180000,54264.48,54272.67,54261.44,54268.82,19.513919
3,1619769240000,54268.82,54304.21,54264.00,54280.33,34.360138
4,1619769300000,54280.32,54304.20,54275.15,54291.79,29.218204


In [13]:
data["tic"] = "BTC/USDT"
data.head()

,date,open,high,low,close,volume,tic
0,1619769060000,54283.73,54286.00,54261.42,54267.52,25.456899,BTC/USDT
1,1619769120000,54267.51,54273.36,54263.74,54264.47,25.162633,BTC/USDT
2,1619769180000,54264.48,54272.67,54261.44,54268.82,19.513919,BTC/USDT
3,1619769240000,54268.82,54304.21,54264.00,54280.33,34.360138,BTC/USDT
4,1619769300000,54280.32,54304.20,54275.15,54291.79,29.218204,BTC/USDT


In [17]:
data["day"] = data["date"].dt.dayofweek
        # convert date to standard string format, easy to filter
data["date"] = data.date.apply(lambda x: x.strftime("%Y-%m-%d"))
data.head()

AttributeError: Can only use .dt accessor with datetimelike values